# Match Kitti Sequence version 22-11

Reads a sequence of stereo images as Frame. The Frames are scanned to identify FramePoints that are pixels that are at the top or botrom of vertical lines in the image. A patch is taken around each of the FramePoints in the left hand image, to match these to a patch in the right hand image and estimate the disparity/distance of the framePoint.

A Sequence is created in which the first Frame is the KeyFrame. It's initial FramePoints are culled to keep only the points for which there is a confident estimate of its depth. The consecutive frames are used to track the KeyFrame's remaining FramePoints. Only a limited number of all framePoints will be found in the consecutive frames, mostly because nearby objects become larger quickly as we get closer. The matching FramePoints are given the same FramePointId as the corresponding point in the KeyFrame and its depth is estimated.

The results are dumped into a folder 'dump', containing files 0.txt, 1.txt, etc. 0.txt contains the FramePoints as observed from the first frame, 1.txt form the second frame, etc. In these files, each row contains a single FramePoint with the values FramePointID, X, Y, Depth.

In [2]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
os.environ['STEREO_CONFIDENCE'] = '1.6'
os.environ['SEQUENCE_CONFIDENCE'] = '1.6'
#os.environ['PATCH_SIZE'] = '17'
FRAMECOUNT = 1101

from pyurb.urb_kitti import *
%matplotlib inline
import numpy as np

ModuleNotFoundError: No module named 'settings'

Read the first 10 frames from KITTI sequence 00.

In [ ]:
%%time
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/01/image_2'
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/01/image_3'

frame = []
for frameid in range(FRAMECOUNT):
    filename = '%06d.png'%(frameid)
    left_frame = Frame(LEFTDIR + '/' + filename, RIGHTDIR)
    frame.append(left_frame)

In [ ]:
%%bash
ls /data/urbinn/datasets/kitti/sequences/01/image_2 | wc -l


Inspect the FramePoints for the first frame

We can also manually compute the Depth of the points and show these

Turn the list of frames into a sequence. The first frame is automatically taken as the KeyFrame and its FramePoints are rerieved, depth is estimated, only the most confident points are kept, and they are tracked in the other frames.

In [ ]:
seq = Sequence()
for f in ProgressBar()(frame):
    seq.add_frame(f)

In [ ]:
show2(draw_frame_depth(frame[0]), draw_frame_depth(frame[1]))

Show only the ID's in frame[0] that are actually in frame[1], to avoid clutter in the image.

In [2]:
#kp0 = [kp for kp in frame[0].get_observations() if kp.get_mappoint() in { p.get_mappoint() for p in frame[1].get_observations() }]
#kp1 = [kp for kp in frame[1].get_observations() if kp.has_mappoint()]
#frame[1].compute_depth()
#show2(draw_observations_depth(kp0), draw_observations_depth(kp1))

In [3]:
frame[5].compute_depth()
frame[5].filter_not_useful()
frame[6].compute_depth()
frame[6].filter_not_useful()
obs1 = [o for o in frame[5].get_observations() ]
obs2 = [o for o in frame[6].get_observations() ]

NameError: name 'frame' is not defined

In [ ]:
show2(draw_observations_depth(obs1), draw_observations_depth(obs2))

In [ ]:
frame[5].compute_depth()
a = [ (o.get_depth(), o.confidence) for o in frame[5].get_observations() if o.confidence is not None and o.confidence > 1.4 ]
len(a)
#show2(draw_frame(frame[5]), draw_frame(frame[5].get_right_frame()))

In [ ]:
pl = [ o for o in frame[5].get_observations() if o.confidence is not None and o.confidence > 1.4 ]
show(draw_observations_depth(pl))

In [ ]:
show2(draw_frame(frame[0]), draw_frame(f.get_right_frame()))

In [ ]:
f = frame[0]

In [ ]:
f.compute_depth()

In [ ]:
show(draw_observations_depth(frame[0].get_observations()[:100]))